In [1]:
''' Set the environment  '''
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem,Draw
from rdkit.Chem import MCS
import pandas as pd
import numpy as np
import copy,re
from rdkit.Chem.Scaffolds.MurckoScaffold import GetScaffoldForMol,MakeScaffoldGeneric,MurckoScaffoldSmiles,MurckoScaffoldSmilesFromSmiles
from my_toolset.my_utils import get_mol,compute_FP,canonic_smiles,mapper,weight
from my_toolset.drawing_utils import show_mols
import os,sys
from utils.common_utils import mapper,csvToExcel,get_mol,compute_FP,canonic_smiles,float_row,heavy_atm_smiles,kekulize_smi,sort_mol_sim_df
from utils.sarm_utils import get_core

from IPython.display import display, SVG, display_svg
import seaborn as sns
from matplotlib import pyplot
from pathlib import Path
import glob
from functools import partial
from pandarallel import pandarallel
n_jobs=40
pandarallel.initialize(nb_workers=n_jobs)

import math
import itertools
import matplotlib.pyplot as plt
import plotly.express as px
import json

from utils.dual_target_utils import kekulize_smi,remove_dummy,get_RGrps_dummy,get_frag,combine_CombTab_Fround1,replace_nH,add_singleBond_dummy,match_frag,get_activity_info,add_singleBond_dummy,get_full_core,is_number,is_parent,is_child,remove_dummy,real_sonNode,remove_ionizationForm,find_children_single,find_children,find_parents_single,find_parents,if_root,find_root,smiles2shapeSmarts,scaffoldFamily,scaffold_info,get_full_core,get_atomRingInfo,match_core,get_atomPos

/shared/data/jay.zhang/anaconda3/envs/default37/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The rdkit.Chem.MCS module is deprecated; please use rdkit.Chem.rdFMCS instead.
  after removing the cwd from sys.path.


INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


#######################################################################################
       ###############   BELOW IS JAK SECTION     #############################
#######################################################################################

In [2]:
'''  Create SARM JAK (High selectivity and activity)  '''
''' Test of MLP program  '''
''' Create normal SARM table 
Can be based on the Scaffold or the full SMILES
'''
from rdkit.Chem.Scaffolds.MurckoScaffold import GetScaffoldForMol,MakeScaffoldGeneric,MurckoScaffoldSmiles,MurckoScaffoldSmilesFromSmiles
from IPython.display import display
import pandas as pd
from rdkit import Chem
import os,sys
sys.path.append('..')
from utils.common_utils import mapper,csvToExcel
from utils.sarm_utils import fragmentize,create_SARM,create_SARM_MLP
from my_toolset.my_utils import canonic_smiles,df_valid

act_csv_file="/shared/data/jay.zhang/Codes/autosarm/CDK2/cdk2_cdac.csv"
target="CDK12_ic50"
value_col=f"CDK1/2_ic50"

save_folder=f"DualTarget_JAKAMPK/ScaffoldSARM/{target}_SMILES"
# act_csv_file="/public/home/zhangjie/Project_support/AIXB-5/Selectivity_Jan16/MedChemAct_Cano_Sele_JAN16.csv"
# save_folder="/public/home/zhangjie/Projects/autosarm/Sele78_Feb13/seleIdx20TLR820/Scaffold_All"
df_sele=pd.read_csv(act_csv_file)
df_sele=df_valid(df_sele,row_smi='SMILES')
df_sele[value_col]=df_sele[value_col].parallel_apply(is_number)
df_sele=pd.DataFrame(df_sele[df_sele[value_col]!=''])

if 'Cano_SMILES' not in df_sele.columns:
    df_sele['Cano_SMILES']=df_sele['SMILES'].parallel_apply(canonic_smiles)
df_sele['Scaffold']=[MurckoScaffoldSmilesFromSmiles(ismi) for ismi in df_sele["Cano_SMILES"]]
# df_sele.to_csv(act_csv_file, index=None)
act_CPDs=df_sele['Scaffold']

print(''' Fragment the scaffold! ''')
df_round1, df_round2=fragmentize(act_CPDs, n_jobs=40, drop_duplicate=False, pos_args={'RR':True, 'nRnR':True})
print("Creating SARM!")
df_table_info_left, df_table_info_right, df_table_info_combine=create_SARM(df_round1, df_round2, df_sele, save_folder=save_folder, smi_col="Scaffold", value_col=value_col, minimum_count=5, csv2excel=True,cal_table_stats=True, n_jobs=40)

 Fragment the scaffold! 
Creating SARM!
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Loading round1 fragments
Loading round2 fragments
Working on row: 0
Working on row: 1
Working on row: 2
Working on row: 3
Working on row: 4
Aromatic atom found! 1
Null molecule provided
Null molecule provided
Processing Table_4, size: (3, 5), count: 15


[13:27:07] SMILES Parse Error: syntax error while parsing: 10
[13:27:07] SMILES Parse Error: Failed parsing SMILES '10' for input: '10'
[13:27:07] SMILES Parse Error: syntax error while parsing: 10
[13:27:07] SMILES Parse Error: Failed parsing SMILES '10' for input: '10'
[13:27:08] SMILES Parse Error: syntax error while parsing: 10
[13:27:08] SMILES Parse Error: Failed parsing SMILES '10' for input: '10'
[13:27:08] SMILES Parse Error: syntax error while parsing: 10
[13:27:08] SMILES Parse Error: Failed parsing SMILES '10' for input: '10'


Null molecule provided
Null molecule provided
Processing Table_4, size: (6, 5), count: 30
frag_cols: Index(['*CC1CCN(c2ccc(C3CCC(=O)NC3=O)cc2)CC1'], dtype='object')
Null molecule provided
Null molecule provided


[13:27:08] SMILES Parse Error: syntax error while parsing: 6
[13:27:08] SMILES Parse Error: Failed parsing SMILES '6' for input: '6'
[13:27:08] SMILES Parse Error: syntax error while parsing: 6
[13:27:08] SMILES Parse Error: Failed parsing SMILES '6' for input: '6'


Working on row: 5
Aromatic atom found! 1
Null molecule provided
Null molecule provided
Processing Table_5, size: (2, 5), count: 10


...

Null molecule provided
Null molecule provided
Processing Table_93, size: (2, 5), count: 10
frag_cols: Index(['*c1ccc(C2CCC(=O)NC2=O)cc1'], dtype='object')
Null molecule provided
Null molecule provided
Working on row: 94
Aromatic atom found! 14


[13:28:01] SMILES Parse Error: syntax error while parsing: 2
[13:28:01] SMILES Parse Error: Failed parsing SMILES '2' for input: '2'
[13:28:01] SMILES Parse Error: syntax error while parsing: 2
[13:28:01] SMILES Parse Error: Failed parsing SMILES '2' for input: '2'
[13:28:01] SMILES Parse Error: syntax error while parsing: 4
[13:28:01] SMILES Parse Error: Failed parsing SMILES '4' for input: '4'
[13:28:01] SMILES Parse Error: syntax error while parsing: 4
[13:28:01] SMILES Parse Error: Failed parsing SMILES '4' for input: '4'


Null molecule provided
Null molecule provided
Processing Table_94, size: (3, 5), count: 15


[13:28:02] SMILES Parse Error: syntax error while parsing: 4
[13:28:02] SMILES Parse Error: Failed parsing SMILES '4' for input: '4'
[13:28:02] SMILES Parse Error: syntax error while parsing: 4
[13:28:02] SMILES Parse Error: Failed parsing SMILES '4' for input: '4'
[13:28:02] SMILES Parse Error: syntax error while parsing: 3
[13:28:02] SMILES Parse Error: Failed parsing SMILES '3' for input: '3'
[13:28:02] SMILES Parse Error: syntax error while parsing: 3
[13:28:02] SMILES Parse Error: Failed parsing SMILES '3' for input: '3'


Null molecule provided
Null molecule provided
Processing Table_94, size: (2, 5), count: 10
frag_cols: Index(['*C1CCN(c2ccc(C3CCC(=O)NC3=O)cc2)CC1'], dtype='object')
Null molecule provided
Null molecule provided
Working on row: 95


In [13]:
'''  Combine combined table and fragment round 1 table   '''

'''  ---JAK---  Construct fragment library: Combine Frag_round1 and Frag_round2  '''
combTable_file="/public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/ScaffoldSARM/JAKA30S10_SMILES/Combine_Table_info.csv"
FRound1_file="/public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/ScaffoldSARM/JAKA30S10_SMILES/Frag_round1_count.csv"
file_act="/public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/Data/JAK_Act30Sel10.csv"
infoCols=["JAK1","JAK1ToJAK2"] 
get_activity_info_p=partial(get_activity_info, file_act=file_act, cols=infoCols)
df_comb=combine_CombTab_Fround1(combTable_file, FRound1_file)
actInfo=df_comb['SMILES'].parallel_apply(get_activity_info_p)
actCols=[]
for idx,icol in enumerate(infoCols):
    actCols+=[f'{icol}_mean',f'{icol}_std',f'{icol}_median']
actInfo=pd.DataFrame(actInfo.to_list())
actInfo.columns=actCols
for icol in actCols:
    df_comb[icol]=actInfo[icol].to_list()
df_comb.to_csv('/public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/Data/JAK1_CombTable_Round1_actInfo.csv', index=None)

In [30]:
''' --JAK--- Put the scaffolds into different groups based on similarity and parents info '''

df_comb=pd.read_csv('/public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/Data/JAK1_CombTable_Round1_actInfo.csv')
df_comb['noDumSmi']=df_comb['SMILES'].parallel_apply(remove_dummy)
df_comb['Wt']=df_comb['SMILES'].parallel_apply(weight)
df_comb=df_comb.sort_values('Wt', ascending=False)  ## make the large scaffold in the front
scaff_NoDumScaff_dict,NoDumScaff_scaff_dict=scaffold_info(df_comb['SMILES'])#(["*Oc1nc2nc(*)ccc2[nH]1","*Oc1nc2nc(-c3ccc(*)cc3)ccc2[nH]1",'*c1ccc(-c2ccc(*)cc2)cc1',"*Oc1nc2nc(*)cnc2[nH]1"])
scaffoldsNodum=list(scaff_NoDumScaff_dict.values())
scaffoldFamily_p=partial(scaffoldFamily, scaffolds=scaffoldsNodum)
scafFamilys=mapper(40)(scaffoldFamily_p,scaffoldsNodum)
scafFam_dict={}
for idx,iscaf in enumerate(scaffoldsNodum):
    scafFam_dict[scaffoldsNodum[idx]]=scafFamilys[idx]
print(f"scafFam_dict= {scafFam_dict}")

'''  Assign Groups  '''
scafFam_dict_keys=list(scafFam_dict.keys())
for idx,ikey in enumerate(scafFam_dict_keys):
    if ikey in scafFam_dict.keys():
        all_members=list(scafFam_dict[ikey])
        new_members=list(scafFam_dict[ikey])
        count=0  ## to make the loop stable
        while len(new_members)>0:
            count+=1
            if count>1000: break
            tmp_members=[]
            for jkey in new_members:
                if jkey in scafFam_dict.keys() and jkey != ikey:
                    tmp_members+=scafFam_dict[jkey]
                    del scafFam_dict[jkey]
            tmp_members=list(set(tmp_members))
            new_members=list(set(tmp_members)-set(all_members))  
            all_members=list(set(all_members+tmp_members))  
        scafFam_dict[ikey]= all_members   
groupInfo_dict={}
for idx,iKey in enumerate(scafFam_dict.keys()):
    for jKey in scafFam_dict[iKey]:
        groupInfo_dict[jKey] = idx
for ismi in df_comb["noDumSmi"]:
    if ismi not in groupInfo_dict.keys():
        groupInfo_dict[ismi]=-1
df_comb['ScaffoldGroup']=df_comb.apply(lambda x: groupInfo_dict[x['noDumSmi']], axis=1)
df_comb=df_comb.sort_values(by=['ScaffoldGroup','Wt'], ascending=True)
df_comb.to_csv('/public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/Data/JAK1_CombTable_Round1_actInfo.csv', index=None)


scafFam_dict= {'CCOC(=O)N1CCCC(n2ccc(Nc3ccccc3)n2)C1': ['CCOC(=O)N1CCCC(n2ccc(Nc3ccccc3)n2)C1', 'COC(=O)N1CCCC(n2ccc(Nc3ccccc3)n2)C1', 'OC(=O)N1CCCC(n2ccc(Nc3ccccc3)n2)C1', 'C(=O)N1CCCC(n2ccc(Nc3ccccc3)n2)C1', 'CN1CCCC(n2ccc(Nc3ccccc3)n2)C1', 'c1ccc(Nc2ccn(C3CCCNC3)n2)cc1', 'CCOC(=O)N1CCCC(n2ccc(N)n2)C1', 'COC(=O)N1CCCC(n2ccc(N)n2)C1', 'CCOC(=O)N1CCCC(n2cccn2)C1', 'OC(=O)N1CCCC(n2ccc(N)n2)C1', 'COC(=O)N1CCCC(n2cccn2)C1', 'OC(=O)N1CCCC(n2cccn2)C1', 'C(=O)N1CCCC(n2ccc(N)n2)C1', 'CN1CCCC(n2ccc(N)n2)C1', 'C(=O)N1CCCC(n2cccn2)C1', 'Nc1ccn(C2CCCNC2)n1', 'Nc1ccn(C2CCCNC2)n1', 'CN1CCCC(n2cccn2)C1', 'c1ccc(Nc2cc[nH]n2)cc1', 'c1cccc(Nc2cc[nH]n2)c1', 'c1ccc(Nc2cc[nH]n2)cc1', 'c1ccn(C2CCCNC2)n1', 'c1ccn(C2CCCNC2)n1', 'c1ccc(N)cc1', 'c1cccc(N)c1', 'Nc1cc[nH]n1', 'c1ccccc1', 'c1ccccc1', 'c1cc[nH]n1'], 'CCOC(=O)N1CCC(n2ccc(Nc3ccccc3)n2)CC1': ['CCOC(=O)N1CCC(n2ccc(Nc3ccccc3)n2)CC1', 'COC(=O)N1CCC(n2ccc(Nc3ccccc3)n2)CC1', 'OC(=O)N1CCC(n2ccc(Nc3ccccc3)n2)CC1', 'C(=O)N1CCC(n2ccc(Nc3ccccc3)n2)CC1', 'CN1CC

In [4]:
'''  ---JAK---  Atom density in the pocket  '''

ifGetAtomPos=0
gridAtom=0
pymolPse=1

work_dir="/public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/Data/JAK_Act_Dock"  ## Dock dir of active CPDs
sdfFiles=glob.glob(f"{work_dir}/sdfs/*.sdf")

'''  Get all the position of atoms  ''' 
if ifGetAtomPos:
    atomPos_list=[]     
    for isdf in sdfFiles:
        atomPos=get_atomPos(isdf)
        for iatmPos in atomPos:
            atomPos_list.append(iatmPos)
    df_ActAtomPos=pd.DataFrame(atomPos_list,columns=['molName','atomIndex','X','Y','Z','AtomicNum','FormalCharge','IsAromatic','IsInRing'])
    df_ActAtomPos.to_csv(f"{work_dir}/JAK_actSet_atomPos.csv", index=None)
    print(f"Total atoms in the active set:  {len(atomPos_list)}.")

'''  Put the atoms into grids  ''' 
if gridAtom:
    grid_parameters={}  ## to save the grid parameters
    df_ActAtomPos=pd.read_csv(f"{work_dir}/JAK_actSet_atomPos.csv")
    df_ActAtomPos[['X','Y','Z']]=df_ActAtomPos[['X','Y','Z']].round(1)
    xyzMax=df_ActAtomPos[['X','Y','Z']].max()
    xyzMin=df_ActAtomPos[['X','Y','Z']].min()
    grid_parameters['xyzMax']=xyzMax.to_dict()
    grid_parameters['xyzMin']=xyzMin.to_dict()
    print(f"xyzMax= {xyzMax}")
    print(f"xyzMin= {xyzMin}")
    cellWidth=1
    grid_parameters['cellWidth']=cellWidth
    xyzRange=(xyzMax-xyzMin)/cellWidth
    for iax in ['X','Y','Z']:
        xyzRange[iax]=int(math.ceil(xyzRange[iax]))
    xList=[xyzMin['X']+cellWidth*i for i in range(int(xyzRange['X']))]
    yList=[xyzMin['Y']+cellWidth*i for i in range(int(xyzRange['Y']))]
    zList=[xyzMin['Z']+cellWidth*i for i in range(int(xyzRange['Z']))]
    xyzIndex=list(itertools.product(xList,yList,zList))
    df_atmCount=pd.DataFrame(xyzIndex,columns=['X','Y','Z'])
    df_atmCount[['X','Y','Z']]=df_atmCount[['X','Y','Z']].astype(str)
    df_atmCount.set_index(['X','Y','Z'], drop=False, inplace=True)
    df_atmCount['Count_Carbon']=0  ## All Carbon
    df_atmCount['Count_ac']=0  ## All Carbon
    df_atmCount['Count_C']=0  ## All Carbon
    df_atmCount['Count_Oxygen']=0  ## All Carbon
    df_atmCount['Count_ao']=0  ## All Carbon
    df_atmCount['Count_O']=0  ## All Carbon
    df_atmCount['Count_Nitrogen']=0  ## All Carbon
    df_atmCount['Count_an']=0  ## All Carbon
    df_atmCount['Count_N']=0  ## All Carbon
    df_atmCount['Count_All']=0   ## All Atom
    for idx,irow in df_ActAtomPos.iterrows():
        ixPos=int((irow['X']-xyzMin['X'])/cellWidth)*cellWidth+xyzMin['X']
        ixPos=str(ixPos)
        iyPos=int((irow['Y']-xyzMin['Y'])/cellWidth)*cellWidth+xyzMin['Y']
        iyPos=str(iyPos)
        izPos=int((irow['Z']-xyzMin['Z'])/cellWidth)*cellWidth+xyzMin['Z']
        izPos=str(izPos)
        try:
            tmp=df_atmCount.loc[(ixPos,iyPos,izPos),'Count_All']
        except Exception as e:
            print(e)
            continue
        df_atmCount.loc[(ixPos,iyPos,izPos),'Count_All']+=1
        if irow['AtomicNum']==6:
            df_atmCount.loc[(ixPos,iyPos,izPos),'Count_Carbon']+=1
            if irow['IsAromatic']==1:
                df_atmCount.loc[(ixPos,iyPos,izPos),'Count_ac']+=1
            else:
                df_atmCount.loc[(ixPos,iyPos,izPos),'Count_C']+=1
        if irow['AtomicNum']==7:
            df_atmCount.loc[(ixPos,iyPos,izPos),'Count_Nitrogen']+=1
            if irow['IsAromatic']==1:
                df_atmCount.loc[(ixPos,iyPos,izPos),'Count_an']+=1
            else:
                df_atmCount.loc[(ixPos,iyPos,izPos),'Count_N']+=1
        if irow['AtomicNum']==8:
            df_atmCount.loc[(ixPos,iyPos,izPos),'Count_Oxygen']+=1
            if irow['IsAromatic']==1:
                df_atmCount.loc[(ixPos,iyPos,izPos),'Count_ao']+=1
            else:
                df_atmCount.loc[(ixPos,iyPos,izPos),'Count_O']+=1
                
    df_atmCount=df_atmCount[df_atmCount['Count_All']>0]
    df_atmCount.to_csv(f"{work_dir}/JAK_actSet_atomPos_gridCount.csv", index=None)
    with open(f"{work_dir}/JAK_actSet_atomPos_gridCount_grid.json", "w") as outfile:
        json.dump(grid_parameters, outfile)
    
    fig=plt.figure()
    ax=fig.add_subplot(111,projection='3d')
    # im=ax.scatter(df_atmCount['X'],df_atmCount['Y'],df_atmCount['Z'],s=df_atmCount['Count_All'])
    fig=px.scatter_3d(df_atmCount,x='X',y='Y',z='Z',size='Count_All')
    fig.show()

if pymolPse==1:
    from importlib import reload
    import pymol
    import pandas as pd
    from pymol import cmd, stored
    reload(pymol)
    
    # pymol.finish_launching() 
    # pl = pymol2.PyMOL()
    # pl.start()
    # cmd=pl.cmd
    
    df_ActAtomPos=pd.read_csv(f"{work_dir}/JAK_actSet_atomPos_gridCount.csv")
    countCols=['Count_All','Count_Carbon','Count_ac','Count_C','Count_Nitrogen','Count_an','Count_N','Count_Oxygen','Count_ao','Count_O']
    
    countColMax=df_ActAtomPos[countCols].max().max()
    for icountCol in countCols:
        # countColMax=df_ActAtomPos[icountCol].max()
        df_ActAtomPos[icountCol]=df_ActAtomPos[icountCol]/countColMax
        
    greyColors=['tin','black','zinc','Grey50']
    orangeColors=['lightorange','brightorange','olive','deepolive','orange']
    blueColors=['lightblue','slate','tv_blue','skyblue','blue']
    redColors=['salmon','deepsalmon','tv_red','raspberry','red']
    # pymol.finish_launching()
    for idx,irow in df_ActAtomPos.iterrows():
        iX=irow['X']
        iY=irow['Y']
        iZ=irow['Z']
        
        if irow['Count_All']>0:
            objName="Count_All_Site"
            cmd.pseudoatom(objName, name=f"DUM{idx}", pos=[iX, iY, iZ], vdw=irow['Count_All'], color="gray80", label="")
            cmd.show("spheres", objName)
            cmd.color("gray80", objName)
        
        for idxCol,icol in enumerate(['Count_Carbon','Count_C','Count_ac']):
            if irow[icol]>0:
                objName=f"{icol}_Site"
                cmd.pseudoatom(objName, name=f"DUM{idx}", pos=[iX, iY, iZ], vdw=irow[icol], color="", label="")
                cmd.show("spheres", objName)
                cmd.color(greyColors[-idxCol-1], objName)
        for idxCol,icol in enumerate(['Count_Nitrogen','Count_an','Count_N']):
            if irow[icol]>0:
                objName=f"{icol}_Site"
                cmd.pseudoatom(objName, name=f"DUM{idx}", pos=[iX, iY, iZ], vdw=irow[icol], color="", label="")
                cmd.show("spheres", objName)
                cmd.color(blueColors[-idxCol-1], objName)
        for idxCol,icol in enumerate(['Count_Oxygen','Count_ao','Count_O']):
            if irow[icol]>0:
                objName=f"{icol}_Site"
                cmd.pseudoatom(objName, name=f"DUM{idx}", pos=[iX, iY, iZ], vdw=irow[icol], color="", label="")
                cmd.show("spheres", objName)
                cmd.color(redColors[-idxCol-1], objName)
    cmd.bg_color("white")            
    cmd.set("sphere_transparency", 0.3, "all")        
    cmd.center("Count_All_Site")
    cmd.save(f"{work_dir}/JAK_dummy.pse")
    pymol.finish_launching()

 PyMOL not running, entering library mode (experimental)


In [ ]:
'''  Get position of fragments  '''
import math
import itertools
import matplotlib.pyplot as plt
import plotly.express as px
import json

work_dir="/public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/Data/JAK_Act_Dock"

grid_parameters=f"{work_dir}/JAK_actSet_atomPos_gridCount_grid.json"    
act_sdf_folder=f"{work_dir}/sdfs"
sdfFiles=glob.glob(f"{act_sdf_folder}/*.sdf") 
atmCount_file=f"{work_dir}/JAK_actSet_atomPos_gridCount.csv"

motif="c1([nH]cc2)c2cncn1"  ## SMILES of most important fragment, cannot be SMARTS

def get_fragPos(frag,sdf):
    suppl = Chem.SDMolSupplier(sdf)
    mol=suppl[0]
    molNumAtoms=mol.GetNumAtoms()
    molAtoms=mol.GetAtoms()
    atomProps=[]
    if molNumAtoms>150:
        return atomProps
    conf = mol.GetConformer()
    # print(f"Molecule name: {mol.GetProp('_Name')}")
    # print(f"Number of atoms: {mol.GetNumAtoms()}")
    if '*' in frag:
        frag=remove_dummy(frag)
    smartMol=Chem.MolFromSmarts(frag)
    matched=mol.GetSubstructMatches(smartMol)
    if len(matched)<1:
        return atomProps
    smartRI=get_atomRingInfo(frag)
    # print(f"smartRI= {smartRI}")
    molRI=get_atomRingInfo(mol)
    for imatch in matched:
        submolRI=molRI[list(imatch)]
        # print(f"submolRI= {submolRI}")
        diff=np.absolute(smartRI-submolRI).sum()
        # print(matched)
        if diff==0:
            for i in imatch:
                molName=mol.GetProp('_Name')
                ipos=conf.GetAtomPosition(i)
                iAtomicNum=molAtoms[i].GetAtomicNum()
                iFormalCharge=molAtoms[i].GetFormalCharge()
                iIsAromatic=int(molAtoms[i].GetIsAromatic())
                iIsInRing=int(molAtoms[i].IsInRing())
                atomProps.append([molName,i,ipos.x,ipos.y,ipos.z,iAtomicNum,iFormalCharge,iIsAromatic,iIsInRing])
    # print(f"{i}:  {atomProps}")
    return atomProps

def get_frag_gridPos(frag,sdfFiles,atmCount_file,grid_parameters_json):
    with open(grid_parameters_json, "r") as outfile:
        grid_parameters=json.load(outfile)   ## Load grid parameters
        xyzMin=grid_parameters['xyzMin']
        xyzMax=grid_parameters['xyzMax']
        cellWidth=grid_parameters['cellWidth']
    df_atmCount=pd.read_csv(atmCount_file)  ## Keep all the dataframe size are the same
    countCols=['Count_All','Count_Carbon','Count_ac','Count_C','Count_Nitrogen','Count_an','Count_N','Count_Oxygen','Count_ao','Count_O']  
    df_atmCount[countCols]=0
    df_atmCount[['X','Y','Z']]=df_atmCount[['X','Y','Z']].astype(str)
    df_atmCount.set_index(['X','Y','Z'], drop=False, inplace=True)
    atomPos_list=[]
    for isdf in sdfFiles:
        atomPos=get_fragPos(frag,isdf)
        for iatmPos in atomPos:
            atomPos_list.append(iatmPos)
    df_ActAtomPos=pd.DataFrame(atomPos_list,columns=['molName','atomIndex','X','Y','Z','AtomicNum','FormalCharge','IsAromatic','IsInRing'])
    for idx,irow in df_ActAtomPos.iterrows():
        ixPos=int((irow['X']-xyzMin['X'])/cellWidth)*cellWidth+xyzMin['X']
        ixPos=str(ixPos)
        iyPos=int((irow['Y']-xyzMin['Y'])/cellWidth)*cellWidth+xyzMin['Y']
        iyPos=str(iyPos)
        izPos=int((irow['Z']-xyzMin['Z'])/cellWidth)*cellWidth+xyzMin['Z']
        izPos=str(izPos)
        try:
            tmp=df_atmCount.loc[(ixPos,iyPos,izPos),'Count_All']
        except Exception as e:
            print(e)
            continue
        df_atmCount.loc[(ixPos,iyPos,izPos),'Count_All']+=1
        if irow['AtomicNum']==6:
            df_atmCount.loc[(ixPos,iyPos,izPos),'Count_Carbon']+=1
            if irow['IsAromatic']==1:
                df_atmCount.loc[(ixPos,iyPos,izPos),'Count_ac']+=1
            else:
                df_atmCount.loc[(ixPos,iyPos,izPos),'Count_C']+=1
        if irow['AtomicNum']==7:
            df_atmCount.loc[(ixPos,iyPos,izPos),'Count_Nitrogen']+=1
            if irow['IsAromatic']==1:
                df_atmCount.loc[(ixPos,iyPos,izPos),'Count_an']+=1
            else:
                df_atmCount.loc[(ixPos,iyPos,izPos),'Count_N']+=1
        if irow['AtomicNum']==8:
            df_atmCount.loc[(ixPos,iyPos,izPos),'Count_Oxygen']+=1
            if irow['IsAromatic']==1:
                df_atmCount.loc[(ixPos,iyPos,izPos),'Count_ao']+=1
            else:
                df_atmCount.loc[(ixPos,iyPos,izPos),'Count_O']+=1
                
    countCols=['Count_All','Count_Carbon','Count_ac','Count_C','Count_Nitrogen','Count_an','Count_N','Count_Oxygen','Count_ao','Count_O']
    countColMax=df_atmCount[countCols].max().max()
    for icountCol in countCols:
        # countColMax=df_ActAtomPos[icountCol].max()
        df_atmCount[icountCol]=df_atmCount[icountCol]/countColMax
    return  df_atmCount


def frag_position_score(frag,motif=motif,sdfFiles=sdfFiles,atmCount_file=atmCount_file,grid_parameters_json=grid_parameters):
    '''  Fragment position score relative to ref motif  '''
    df_motif=get_frag_gridPos(frag=motif,sdfFiles=sdfFiles,atmCount_file=atmCount_file,grid_parameters_json=grid_parameters_json) 
    df_frag=get_frag_gridPos(frag=frag,sdfFiles=sdfFiles,atmCount_file=atmCount_file,grid_parameters_json=grid_parameters_json) 
    score=(df_motif[countCols].astype(float)*df_frag[countCols].astype(float)).sum().sum()
    return score

df_comb=pd.read_csv('/public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/Data/JAK1_CombTable_Round1_actInfo.csv')  ## Fragment file
df_comb['FragPosScore']=df_comb['SMILES'].parallel_apply(frag_position_score)

df_comb.to_csv('/public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/Data/JAK1_CombTable_Round1_actInfo_poseScore.csv',index=None)
os.system("python /public/home/zhangjie/Projects/r-group-decoration/csv2Excel.py  --csvFile  /public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/Data/JAK1_CombTable_Round1_actInfo_poseScore.csv")


In [5]:
'''   Select the fragments for dual-target compound enumeration  '''
def dummyAtom_number(ismi):
    '''  count the dummy atoms '''
    return ismi.count('*')
def seleForEnum(irow):
    ''' Select the fragments for enum  '''
    wtLim=300
    dummyAtom_count=1
    FragPos_score=5
    if irow['dummyCount']==dummyAtom_count and irow['Wt']<wtLim and irow['FragPosScore']>FragPos_score:
        return 1
    return 0
    
dfPoseScore=pd.read_csv('/public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/Data/JAK1_CombTable_Round1_actInfo_poseScore.csv')
dfPoseScore['dummyCount']=dfPoseScore['SMILES'].apply(dummyAtom_number)
dfPoseScore['SeleEnum']=dfPoseScore.apply(lambda x: seleForEnum(x),axis=1)
dfPoseScore=dfPoseScore.sort_values(by=['SeleEnum','Wt'],ascending=False)
dfPoseScore.to_csv('/public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/Data/JAK1_CombTable_Round1_actInfo_poseScore.csv',index=None)
os.system("python /public/home/zhangjie/Projects/r-group-decoration/csv2Excel.py  --csvFile  /public/home/zhangjie/Projects/autosarm/DualTarget_JAKAMPK/Data/JAK1_CombTable_Round1_actInfo_poseScore.csv")



/public/home/zhangjie/Projects/r-group-decoration/csv2Excel.py:9: DeprecationWarning: The rdkit.Chem.MCS module is deprecated; please use rdkit.Chem.rdFMCS instead.
  from rdkit.Chem import MCS
/public/home/zhangjie/anaconda3/envs/reinvent1/lib/python3.7/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/public/home/zhangjie/anaconda3/envs/reinvent1/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


0